Kenan Stredic

# Recommendation Systems
We will use the surprise library of Python. Details are available at: http://surpriselib.com

We will first work through an example using a built-in dataset and then use a custom one.

First, ensure that you have the library installed and then load the required packages.

In [9]:
!pip install scikit-surprise

  Using cached scikit-surprise-1.1.3.tar.gz (771 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163485 sha256=71a0c68ba47050d570322f1cba59df89f4eebe83d580fffcb88da78975402b33
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [10]:
import io

import numpy as np
import pandas as pd
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import KNNBaseline
from surprise import Dataset
from surprise import get_dataset_dir
from surprise import accuracy
from surprise.model_selection import KFold

For a recommendation system, we require a file containing at least 3 things - userId, itemId, and rating. Any other information is not needed, but can be good for human analysis of results.

Let's load the built in ml-100k dataset that contains movies and ratings.

In [11]:
# Load the movielens-100k dataset (download it if needed),
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [12]:
# Let's see what files come with the dataset
!ls /root/.surprise_data/ml-100k/ml-100k/

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [13]:
# TODO: Show the first 10 lines of the u.data, and u.item files
!head -10 /root/.surprise_data/ml-100k/ml-100k/u.data

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013


## Algorithms
Let's look at some of the algorithms available with the package

In [14]:
?KNNBaseline

The nearest neighbor methods works by searching for neighbors using the utility matrix. Let's create a nearest neighbor first by item and user

In [15]:
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
# we are going to use item-item similarity
sim_options = {'name': 'pearson_baseline', 'user_based': False}
algo = KNNBaseline(sim_options=sim_options)
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [16]:
!head -10 /root/.surprise_data/ml-100k/ml-100k/u.item

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

# Id to Name Lookup
Let's write a small method that will convert id to name, and name to id

In [17]:
def read_item_names():
    """Read the u.item file from MovieLens 100-k dataset and return two
    mappings to convert raw ids into movie names and movie names into raw ids.
    """

    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    name_to_rid = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = line[1]
            name_to_rid[line[1]] = line[0]

    return rid_to_name, name_to_rid

In [18]:
# test this function
rid_to_name, name_to_rid = read_item_names()

In [19]:
rid_to_name["1"]

'Toy Story (1995)'

In [20]:
name_to_rid["Twelve Monkeys (1995)"]

'7'

In [21]:
# Find top 10 movies similar to movie with id 100

movie_inner_id = algo.trainset.to_inner_iid("200")
movie_name = rid_to_name["200"]

# Retrieve inner ids of the nearest neighbors of Toy Story.
movie_neighbors = algo.get_neighbors(movie_inner_id, k=10)

# Convert inner ids of the neighbors into names.
movie_neighbors = (algo.trainset.to_raw_iid(inner_id)
                       for inner_id in movie_neighbors)
movie_neighbors = (rid_to_name[rid]
                       for rid in movie_neighbors)

print()

print('The 10 nearest neighbors of ' + movie_name)
for movie in movie_neighbors:
    print(movie)


The 10 nearest neighbors of Shining, The (1980)
Bonnie and Clyde (1967)
Godfather: Part II, The (1974)
Alien (1979)
Godfather, The (1972)
Raging Bull (1980)
Pulp Fiction (1994)
One Flew Over the Cuckoo's Nest (1975)
Carrie (1976)
Koyaanisqatsi (1983)
His Girl Friday (1940)


Let's now apply the algorithm and figure out it's accuracy

In [22]:
testset = trainset.build_testset()
predictions = algo.test(testset)
# RMSE should be low as we are biased
accuracy.rmse(predictions, verbose=True)  # ~ 0.68 (which is low)

RMSE: 0.4807


0.48071109787164656

Now, let's also try some baseline methods. Follow the code available here:

https://github.com/NicolasHug/Surprise/blob/fa7455880192383f01475162b4cbd310d91d29ca/examples/baselines_conf.py

For more elaborate testing and validation, follow steps mentioned here
https://github.com/NicolasHug/Surprise/blob/fa7455880192383f01475162b4cbd310d91d29ca/examples/grid_search_usage.py

# Assignment

In this part, you will use the dataset that is provided along with the following Kaggle competition

https://www.kaggle.com/arashnic/book-recommendation-dataset


I have uploaded the files for you at

Ratings file - https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Ratings.csv

Books file - https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Books.csv


Follow the steps below to create a recommendation system from this data

In [23]:
# TODO: Read both the data files into Pandas dataframes

In [24]:
# TODO: Answer the following questions:

# How many ratings and how many books are there in the dataset

# Find the top 10 books have received the highest count of ratings. You should output the id of the book, its title, and the count of ratings received.



In [25]:
# TODO: Important - You may not be able use the whole dataset for model creation, so you need to create a
# smaller sample to proceeed further
# Here is what I did:
# reviews_short = reviews.sample(n = 1000, random_state = 42)
# you can try larger values of n, if the system allows you.

In [26]:
# TODO: Use the data to create a custom dataset in the surprise library
# Steps to do this are: https://surprise.readthedocs.io/en/stable/getting_started.html#use-a-custom-dataset

In [27]:
# TODO: Choose a book at random and use the KNNBasic algorithm to find out its 10 closest neighbors. Do the results make
# sense?

In [28]:
# TODO: Use ParameterGridSearch on the following algorithms and compare their accuracies. You are free to decide
# which specific parameters to use:
# 1. KNNBaseline
# 2. ALS - Baseline
# 3. SGD - Baseline
# 4. SVD
# You should use a cv value of at least 3 and compare the mean accuracy of each of the algorithms
# Comment on whether there is significant differences in the results of the algorithms

In [62]:
import pandas as pd
from surprise import Dataset, Reader, KNNBasic, KNNBaseline, BaselineOnly, SVD, SVDpp, NMF
from surprise.model_selection import cross_validate

# Reading data into Pandas DataFrames
ratings_url = 'https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Ratings.csv'
books_url = 'https://an-utd-course.s3.us-west-1.amazonaws.com/CompDS/Books.csv'

ratings = pd.read_csv(ratings_url)
books = pd.read_csv(books_url)

print(f"Number of ratings: {len(ratings)}")
print(f"Number of books: {len(books)}")

# Finding top 10 books with the highest count of ratings
top_books = ratings['Book-Rating'].groupby(ratings['ISBN']).count().sort_values(ascending=False).head(10)
top_books_info = books.loc[books['ISBN'].isin(top_books.index)]

print("Top 10 books with the highest count of ratings:")
print(top_books_info[['ISBN', 'Book-Title']].merge(top_books, left_on='ISBN', right_index=True))

# Using a smaller sample from the ratings dataset
reviews_short = ratings.sample(n=1000, random_state=42)

# Creating a custom dataset in the surprise library
reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(reviews_short[['User-ID', 'ISBN', 'Book-Rating']], reader)

# Building the full trainset and fitting KNNBasic algorithm
trainset = data.build_full_trainset()
algo = KNNBasic()
algo.fit(trainset)

# Choosing a random book ID and finding its 10 closest neighbors
chosen_book = reviews_short['ISBN'].sample().iloc[0]

book_inner_id = algo.trainset.to_inner_iid(chosen_book)
if book_inner_id is not None and book_inner_id < len(algo.trainset.all_items()):
    book_neighbors = algo.get_neighbors(book_inner_id, k=10)
    book_neighbors = (algo.trainset.to_raw_iid(inner_id) for inner_id in book_neighbors)

    print(f"10 nearest neighbors of book {chosen_book}:")
    for book in book_neighbors:
        print(book)
else:
    print("Chosen book ID not found in the trainset or its inner ID is out of bounds.")

# Define a custom parameter grid search function (since ParameterGridSearch was not working)
def custom_parameter_search(algos, param_grids, data):
    results = {}
    for algo_name, algo_class in algos.items():
        print(f"Running {algo_name}...")
        param_grid = param_grids.get(algo_name, None)

        if param_grid is None:
            print(f"No parameters defined for {algo_name}. Skipping...")
            continue

        best_rmse = float('inf')
        best_params = None

        if algo_name.startswith('BaselineOnly'):
            for params in param_grid:
                algo = algo_class(bsl_options=params.get('bsl_options', {}))
                cv_results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
                mean_rmse = cv_results['test_rmse'].mean()

                if mean_rmse < best_rmse:
                    best_rmse = mean_rmse
                    best_params = params

        elif algo_name == 'SVD':
            for params in param_grid:
                algo = algo_class(n_factors=params.get('n_factors'), n_epochs=params.get('n_epochs'), lr_all=params.get('lr_all'), reg_all=params.get('reg_all'))
                cv_results = cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)
                mean_rmse = cv_results['test_rmse'].mean()

                if mean_rmse < best_rmse:
                    best_rmse = mean_rmse
                    best_params = params

        results[algo_name] = {'best_rmse': best_rmse, 'best_params': best_params}
        print(f"{algo_name} - Best RMSE: {best_rmse:.4f} | Best Params: {best_params}")

    return results



# Defining algorithms and their respective parameter grids for grid search
algos = {
    'KNNBaseline': KNNBaseline,
    'BaselineOnly_ALS': BaselineOnly,
    'BaselineOnly_SGD': BaselineOnly,
    'SVD': SVD,
}

# Define parameter grids for each algorithm
param_grids = {
    'KNNBaseline': [{
        'bsl_options': {'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5},
        'sim_options': {'name': 'pearson_baseline', 'user_based': False}
    }],
    'BaselineOnly_ALS': [{
        'bsl_options': {'method': 'als', 'n_epochs': 10},
    }],
    'BaselineOnly_SGD': [{
        'bsl_options': {'method': 'sgd', 'n_epochs': 20},
    }],
    'SVD': [
        {'n_factors': 50, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02},
        {'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1},
    ]
}

# Run the custom parameter grid search
results = custom_parameter_search(algos, param_grids, data)

<ipython-input-62-a6496079dc13>:10: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(books_url)


Number of ratings: 1149780
Number of books: 271360
Top 10 books with the highest count of ratings:
            ISBN                                       Book-Title  Book-Rating
26    0971880107                                      Wild Animus         2502
118   0671027360                              Angels &amp; Demons          586
305   044023722X                                  A Painted House          647
356   0142001740                          The Secret Life of Bees          615
408   0316666343                        The Lovely Bones: A Novel         1295
522   0312195516              The Red Tent (Bestselling Backlist)          723
748   0385504209                                The Da Vinci Code          883
1105  0060928336  Divine Secrets of the Ya-Ya Sisterhood: A Novel          732
1922  067976402X                           Snow Falling on Cedars          614
Computing the msd similarity matrix...
Done computing similarity matrix.
10 nearest neighbors of book 037319550